<h2><center>Importation des package</center></h2>
<center>(nombre des lucioles "nl", max bound "MB",
min bound "mb",nombre des variables "nv",..</center>

In [13]:
print("*************************************************** Importation des packages***************************************************")
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, BatchNormalization, Activation, Conv2D, MaxPooling2D, Dropout
import numpy
from keras.preprocessing import image
from PIL import Image
import numpy as np
import sys
import os
import csv
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from numpy import expand_dims
from tensorflow.python.eager.function import Function
import numpy as np
import math
import operator
import time as time
import psutil
import numpy
import matplotlib.pyplot as plt
print("* list des packages: ")
print("keras,numpy,PIL,SYS,OS,CSV,tensorflow,MATH,OPERATOR,TIME,PSUTIL,matplotlib")

*************************************************** Importation des packages***************************************************
* list des packages: 
keras,numpy,PIL,SYS,OS,CSV,tensorflow,MATH,OPERATOR,TIME,PSUTIL,matplotlib


In [15]:
import numpy
import matplotlib.pyplot as plt
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

<h2><center>initialisation des paramères</center></h2>
<center>nombre des lucioles , max bound ,min bound,nombre des variables ,..</center>

In [33]:
print("debut.......")

# #DEFINITION OF CNN PARAMETERS
batch_size = 5
kernel_size = (4,)
stride = 1
# #EPOCHS AND FILTERS ARE DEFINED BY PARTICLES
#
#DEFINITION OF firefly PARAMETERS
numberFirefly = 6
iterations = 4
#
minBound = numpy.ones(2)  # MIN BOUND FOR TWO DIMENSIONS IS 1
maxBound = numpy.ones(2)  # ONLY INITIALIZATION
maxBound[0] = 150  # MAX NUMBER OF FILTERS
maxBound[1] = 300  # MAX NUMBER OF EPOCHS
bounds = (minBound, maxBound)
print("bound:",bounds)
print("****************************************************************************************************************")

debut.......
bound: (array([1., 1.]), array([150., 300.]))
****************************************************************************************************************


<h3><center>generation des populations</center></h3>
<center><strong> 1/  génération des positions des lucioles </strong> <br>générer  des lucioles aléatoirement "nl" firefly chaque firefly contient "nv" variables compris entre "MB" et "mb" </center>

In [34]:
print("generation des populations")
from numpy.random import random_sample
def generate_population(population_size, problem_dim, min_bound, max_bound):
    error = 1e-10
    data = (max_bound + error - min_bound) * random_sample((population_size, problem_dim)) + min_bound
    #data[data > max_bound] = max_bound
    return data

fireflies=generate_population(numberFirefly, 2, minBound, maxBound)
fireflies = fireflies.astype(int)
print("population :")
print(fireflies)
print("*******************************************************************************************************************")

generation des populations
population :
[[ 68 174]
 [  3 168]
 [ 12 104]
 [ 63  96]
 [ 39 236]
 [ 31  33]]
*******************************************************************************************************************


# DATA

In [18]:
import pandas as pd
export_dataframe_1 = pd.read_excel('export_dataframe.xlsx')

In [19]:
liste_foruni=list(export_dataframe_1['Code Fournisseur'].unique())
code_num=[i for i in range(len(liste_foruni))]
code={}
for id,cod in zip(liste_foruni,code_num):
    code[id]=cod

In [20]:
def get_id_fourni(cod):
    return code[cod]
export_dataframe_1['id_Fournisseur']=export_dataframe_1['Code Fournisseur'].apply(get_id_fourni,1)


In [21]:
export_dataframe_2=export_dataframe_1.set_index(['Date Opération'])
export_dataframe_3=export_dataframe_1.set_index(['Date Opération'])
export_dataframe_2=export_dataframe_2.drop(['Ligne', 'Dossier', 'Poids Taxable', 'Catégorie', 'Date présentation',
       'Devise Prévision', 'Prévision', 'Devise Réalisation','N° Achat', 'Date Achat', 'Code Fournisseur', 'prev_resultat',
       'cout_analys'],1)

<ipython-input-21-92c033489a2e>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  export_dataframe_2=export_dataframe_2.drop(['Ligne', 'Dossier', 'Poids Taxable', 'Catégorie', 'Date présentation',


In [22]:
split_date = '2021-12-31'

df_training = export_dataframe_2.loc[export_dataframe_2.index <= split_date]
df_test = export_dataframe_2.loc[export_dataframe_2.index > split_date]
print(f"{len(df_training)} days of training data \n {len(df_test)} days of testing data ")

#df_training.to_csv('datasets/training.csv')
#df_test.to_csv('datasets/test.csv')
df_training.head(3)

620084 days of training data 
 32159 days of testing data 


,Réalisation,id_Fournisseur
Date Opération,,
2017-01-02,44.571429,0
2017-01-02,9.523810,1
2017-01-02,3.904762,2


In [23]:
# ADD time features to our model
def create_time_features(df, target=None):
    """
    Creates time series features from datetime index
    """
    df['date'] = df.index
    df['hour'] = df['date'].dt.hour
    df['dayofweek'] = df['date'].dt.dayofweek
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofyear'] = df['date'].dt.dayofyear
    df['sin_day'] = np.sin(df['dayofyear'])
    df['cos_day'] = np.cos(df['dayofyear'])
    df['dayofmonth'] = df['date'].dt.day
    df['weekofyear'] = df['date'].dt.weekofyear
    X = df.drop(['date'], axis=1)
    if target:
        y = df[target]
        X = X.drop([target], axis=1)
        return X, y


In [24]:
from sklearn.preprocessing import StandardScaler
X_train_df, y_train = create_time_features(export_dataframe_2, target='Réalisation')
X_test_df, y_test = create_time_features(df_test, target='Réalisation')
scaler = StandardScaler()
scaler.fit(X_train_df)  # No cheating, never scale on the training+test!
X_train = scaler.transform(X_train_df)
X_test = scaler.transform(X_test_df)

X_train_df = pd.DataFrame(X_train, columns=X_train_df.columns)
X_test_df = pd.DataFrame(X_test, columns=X_test_df.columns)

<ipython-input-23-3d74ea191bd7>:16: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df['weekofyear'] = df['date'].dt.weekofyear
<ipython-input-23-3d74ea191bd7>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = df.index
<ipython-input-23-3d74ea191bd7>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['hour'] = df['date'].dt.hour
<ipython-input-23-3d74ea191bd7>:8: SettingWithCopyWarning: 
A value is trying to be set 

In [25]:
# For our dl model we will create windows of data that will be feeded into the datasets, for each timestemp T we will append the data from T-7 to T to the Xdata with target Y(t)
BATCH_SIZE = 64
BUFFER_SIZE = 100
WINDOW_LENGTH = 24


def window_data(X, Y, window=7):
    '''
    The dataset length will be reduced to guarante all samples have the window, so new length will be len(dataset)-window
    '''
    x = []
    y = []
    for i in range(window-1, len(X)):
        x.append(X[i-window+1:i+1])
        y.append(Y[i])
    return np.array(x), np.array(y)

In [26]:
import tensorflow as tf 
# Since we are doing sliding, we need to join the datasets again of train and test
X_w = np.concatenate((X_train, X_test))
y_w = np.concatenate((y_train, y_test))

X_w, y_w = window_data(X_w, y_w, window=WINDOW_LENGTH)
X_train_w = X_w[:-len(X_test)]
y_train_w = y_w[:-len(X_test)]
X_test_w = X_w[-len(X_test):]
y_test_w = y_w[-len(X_test):]

# Check we will have same test set as in the previous models, make sure we didnt screw up on the windowing
print(f"Test set equal: {np.array_equal(y_test_w,y_test)}")

train_data = tf.data.Dataset.from_tensor_slices((X_train_w, y_train_w))
train_data = train_data.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

Test set equal: True


In [27]:
val_data = tf.data.Dataset.from_tensor_slices((X_test_w, y_test_w))
val_data = val_data.batch(BATCH_SIZE).repeat()


<center><strong> 2/ genétarion des luminisotés des firefly</strong>  <br> initialiser la luminosité 'I' des chaque firefly généré par le taux de perte de modèle basé sur les paramètre de cette firefly</strong></center> 

In [23]:
print('*******************************************declaration de cnn_covid19**********************************************')
print("cette fonction va entrainer le modèle crée par les nombre des epochs et filters de chaque firefly et retourner le taux"
      "de perte de cette modèle après le test de cette modèle sur un ensemble de test")

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.metrics import mean_squared_error
from math import sqrt

def Lstm_optm(epochs, filters,save=False):
    
    dropout = 0.0
    simple_lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(
        128, input_shape=X_train_w.shape[-2:], dropout=dropout),
    tf.keras.layers.Dense(128),
    tf.keras.layers.Dense(128),
    tf.keras.layers.Dense(1)
    ])
    
    simple_lstm_model.compile(optimizer='rmsprop', loss='mse')



    EVALUATION_INTERVAL = 200
    EPOCHS = epochs

    model_history = simple_lstm_model.fit(train_data, epochs=EPOCHS,
                                          steps_per_epoch=EVALUATION_INTERVAL,
                                          validation_data=val_data, validation_steps=filters)  # ,callbacks=[tensorboard_callback]) #Uncomment this line for tensorboard support


    yhat = simple_lstm_model.predict(X_test_w).reshape(1, -1)[0]
    rmse = sqrt(mean_squared_error(y_test, yhat))

    return rmse
    

*******************************************declaration de cnn_covid19**********************************************
cette fonction va entrainer le modèle crée par les nombre des epochs et filters de chaque firefly et retourner le tauxde perte de cette modèle après le test de cette modèle sur un ensemble de test


In [24]:
print("******************************************déclaration de optimize CNN****************************************************")
print("cette fonction va avoir une luciole contenant le nombre des epochs et des filters généré après on va calculer l'accuracy"
      "de modèle basé sur cette nombre des epochs et filters (en passant ces paramètre a la fonction CNN_COVID) et finalement on va calculer" 
      "le taux de perte en se basant sur l'accuracy et nombre de filter et des epochs")
def LossCNN(firefly):
    try:
        
        print(firefly)
        #RETRIEVE DIMENSIONS OF PARTICLE
        numberFilters = int(firefly[0]) #FLOAT TO INT
        numberEpochs = int(firefly[1])

        #CALL CNN FUNCTION cnn --> RETURN accuracy
        rmse = Lstm_optm(epochs=numberEpochs, filters=numberFilters)

        #APPLY LOST FUNCTION --> THE MAIN OBJECTIVE IS TO MINIMIZE LOSS --> MAXIMIZE ACCURACY AND AT SAME TIME MINIMIZE THE NUMBER OF EPOCHS
                                #AND FILTERS, TO REDUCE TIME AND COMPUTACIONAL POWER
        
        return rmse

    except:
        raise

******************************************déclaration de optimize CNN****************************************************
cette fonction va avoir une luciole contenant le nombre des epochs et des filters généré après on va calculer l'accuracyde modèle basé sur cette nombre des epochs et filters (en passant ces paramètre a la fonction CNN_COVID) et finalement on va calculerle taux de perte en se basant sur l'accuracy et nombre de filter et des epochs


In [25]:
print("****************************************déclaration au Firefly-Iteration***********************************************")
print("cette fonction va boucler sur le nombre des lucioles générer et pour chaque itération on va faire appel "
      "a la fonction LossCNN qui va retourne le taux de perte de modèle de firefly passé en paramètre")
def FireflyIteration(fireflies):
    try:
        numberFirefly = fireflies.shape[0]
        allLosses = [LossCNN(firefly=fireflies[i])for i in range(numberFirefly)]


        return allLosses#NEED TO RETURN THIS PYSWARMS NEED THIS

    except:
        raise

****************************************déclaration au Firefly-Iteration***********************************************
cette fonction va boucler sur le nombre des lucioles générer et pour chaque itération on va faire appel a la fonction LossCNN qui va retourne le taux de perte de modèle de firefly passé en paramètre


In [26]:
print("initialisation...............")
allLosses=np.array([])
allLosses=FireflyIteration(fireflies)


initialisation...............
[86 36]
Epoch 1/36
200/200 [==============================] - 5s 24ms/step - loss: 435724.6250 - val_loss: 425398.8750
Epoch 2/36
200/200 [==============================] - 5s 23ms/step - loss: 506085.0000 - val_loss: 426183.8125
Epoch 3/36
200/200 [==============================] - 5s 26ms/step - loss: 585119.1875 - val_loss: 425143.0625
Epoch 4/36
200/200 [==============================] - 6s 29ms/step - loss: 454336.7500 - val_loss: 427823.2188
Epoch 5/36
200/200 [==============================] - 7s 36ms/step - loss: 359008.1562 - val_loss: 427992.3125
Epoch 6/36
200/200 [==============================] - 8s 40ms/step - loss: 527803.4375 - val_loss: 458802.0000
Epoch 7/36
200/200 [==============================] - 8s 40ms/step - loss: 484538.5938 - val_loss: 436564.8438
Epoch 8/36
200/200 [==============================] - 8s 41ms/step - loss: 633004.9375 - val_loss: 524923.3750
Epoch 9/36
200/200 [==============================] - 8s 41ms/step - loss:

200/200 [==============================] - 17s 83ms/step - loss: 401918.5938 - val_loss: 880560.6875
Epoch 17/114
200/200 [==============================] - 16s 81ms/step - loss: 462593.2812 - val_loss: 926629.8125
Epoch 18/114
200/200 [==============================] - 17s 85ms/step - loss: 421295.1562 - val_loss: 955820.0625
Epoch 19/114
200/200 [==============================] - 17s 87ms/step - loss: 436901.2812 - val_loss: 990872.4375
Epoch 20/114
200/200 [==============================] - 17s 86ms/step - loss: 490977.0938 - val_loss: 1054799.6250
Epoch 21/114
200/200 [==============================] - 17s 83ms/step - loss: 407488.3750 - val_loss: 935031.6250
Epoch 22/114
200/200 [==============================] - 17s 83ms/step - loss: 457893.8125 - val_loss: 913390.2500
Epoch 23/114
200/200 [==============================] - 18s 92ms/step - loss: 490595.8750 - val_loss: 869018.2500
Epoch 24/114
200/200 [==============================] - 16s 82ms/step - loss: 428983.5312 - val_loss

Epoch 88/114
200/200 [==============================] - 16s 81ms/step - loss: 695936.2500 - val_loss: 814042.7500
Epoch 89/114
200/200 [==============================] - 16s 81ms/step - loss: 1092974.0000 - val_loss: 808349.4375
Epoch 90/114
200/200 [==============================] - 17s 83ms/step - loss: 721302.3125 - val_loss: 858562.9375
Epoch 91/114
200/200 [==============================] - 16s 80ms/step - loss: 461066.1562 - val_loss: 807106.3125
Epoch 92/114
200/200 [==============================] - 16s 81ms/step - loss: 823848.7500 - val_loss: 812486.3750
Epoch 93/114
200/200 [==============================] - 16s 81ms/step - loss: 596153.6250 - val_loss: 815472.8125
Epoch 94/114
200/200 [==============================] - 17s 83ms/step - loss: 1219054.0000 - val_loss: 817130.6250
Epoch 95/114
200/200 [==============================] - 17s 83ms/step - loss: 3666562.2500 - val_loss: 806435.0000
Epoch 96/114
200/200 [==============================] - 16s 80ms/step - loss: 4346336

200/200 [==============================] - 20s 98ms/step - loss: 810553.5000 - val_loss: 570960.0625
Epoch 87/144
200/200 [==============================] - 22s 108ms/step - loss: 778788.5000 - val_loss: 573135.1250
Epoch 88/144
200/200 [==============================] - 19s 95ms/step - loss: 691465.3750 - val_loss: 574825.3750
Epoch 89/144
200/200 [==============================] - 21s 106ms/step - loss: 1089674.7500 - val_loss: 573775.2500
Epoch 90/144
200/200 [==============================] - 19s 93ms/step - loss: 716376.0000 - val_loss: 604609.8750
Epoch 91/144
200/200 [==============================] - 21s 105ms/step - loss: 454502.2500 - val_loss: 564647.3125
Epoch 92/144
200/200 [==============================] - 19s 94ms/step - loss: 817386.5625 - val_loss: 574712.5625
Epoch 93/144
200/200 [==============================] - 19s 97ms/step - loss: 598483.1250 - val_loss: 568428.0000
Epoch 94/144
200/200 [==============================] - 19s 94ms/step - loss: 1218817.0000 - val_

In [27]:
print("fin de initialisation des paramère")

fin de initialisation des paramère


In [28]:
print("************************************  voila les paramètre (loss) des firefly a initialisé**********************")
print(allLosses)
for i in range(numberFirefly):
    print("firefly [ epochs = ",fireflies[i][1],", filter = " ,fireflies[i][0], "] ==> Loss (lumunisoté :)",allLosses[i]) 

************************************  voila les paramètre (loss) des firefly a initialisé**********************
[759.2343645719959, 750.620628101023, 751.869245692173, 795.8956073262817, 747.925084446167, 796.4490197497768]
firefly [ epochs =  36 , filter =  86 ] ==> Loss (lumunisoté :) 759.2343645719959
firefly [ epochs =  93 , filter =  65 ] ==> Loss (lumunisoté :) 750.620628101023
firefly [ epochs =  114 , filter =  16 ] ==> Loss (lumunisoté :) 751.869245692173
firefly [ epochs =  30 , filter =  98 ] ==> Loss (lumunisoté :) 795.8956073262817
firefly [ epochs =  144 , filter =  116 ] ==> Loss (lumunisoté :) 747.925084446167
firefly [ epochs =  36 , filter =  6 ] ==> Loss (lumunisoté :) 796.4490197497768


<h3><center>le lancement de l'optimisation A l'aide de l'algorithme Firefly</center></h3>

from IPython.display import Image
Image("DPL3.png",width=1000)

In [29]:
print("********* class Firefly ********")

class Firefly:

    def __init__(self, position,brightness,function):
        self.func = function                         # Choose the benchmark algorithm to run
        self.position = position
        self.brightness = brightness
        #self.update_brightness()

    # the best fit is 0
    def update_brightness(self):
        self.brightness = -self.func(self.position)

********* class Firefly ********


In [30]:
class FireflyOptimizer:

    def __init__(self, obj, **kwargs):
        self.population_size = int(kwargs.get('population_size', 4))
        self.problem_dim = kwargs.get('problem_dim', 2)
        self.min_bound = kwargs.get('min_bound', 1)
        self.max_bound = kwargs.get('max_bound', 30)
        self.generations = kwargs.get('generations', 2)
        self.population = self._population(self.population_size, self.problem_dim, self.min_bound, self.max_bound)
        self.gamma = kwargs.get('gamma', 0.95)  # absorption coefficient
        self.alpha = kwargs.get('alpha', 0.25)  # randomness [0,1]
        self.beta_init = kwargs.get('beta_init', 1)
        self.beta_min = kwargs.get('beta_min', 0.2)
        self.optimization_benchmark = obj
        self.dtime = 0

    @staticmethod
    def _population(population_size, problem_dim, min_bound, max_bound):
        population = []
        for i in range(population_size):
            population.append(Firefly(problem_dim, min_bound, max_bound))
        return population

    def step(self):
        self.population.sort(key=operator.attrgetter('brightness'), reverse=True)
        self._modify_alpha()
        tmp_population = self.population
        for i in range(self.population_size):
            for j in range(self.population_size):
                if self.population[i].brightness > tmp_population[j].brightness:
                    r = math.sqrt(np.sum((self.population[i].position - tmp_population[j].position) ** 2))
                    beta = (self.beta_init - self.beta_min) * math.exp(-self.gamma * r ** 2) + self.beta_min
                    tmp = self.alpha * (np.random.random_sample((1, self.problem_dim))[0] - 0.5) * (
                            self.max_bound - self.min_bound)
                    self.population[j].position = self.check_position(
                        self.population[i].position * (1 - beta) + tmp_population[
                            j].position * beta + tmp)
                    self.population[j].update_brightness()

        # Changing the position of Brightness so the population will shift to other locations

        # self.population[0].position = generate_population(1, self.problem_dim, self.min_bound, self.max_bound)[0]
        # self.population[0].update_brightness()

    # Best placement for each Generation
    def run_firefly(self, population):
        self.population = population
        start = time.time()

        self.res = {}
        for t in range(self.generations):
            self.res[t] = ('Generation %s, best fitness %s' % (t, self.population[0].brightness))
            self.step()
        end = time.time()
        self.dtime = end - start
        self.dtime = float(str(end - start)[:4])
        # """ psg.popup_scrolled('FA Results\n',
        #                  self.res,
        #                 '\nAlgorithm run time: ', self.dtime,
        #                '\nAverage CPU usage: ', psutil.cpu_percent(), size=[50, 30])"""

        self.population.sort(key=operator.attrgetter('brightness'), reverse=True)
        return self.population[0].brightness, self.population[0].position
    def check_position(self, position):
        position[position > self.max_bound] = self.max_bound
        position[position < self.min_bound] = self.min_bound
        return position

    def _modify_alpha(self):
        delta = 1 - (10 ** (-4) / 0.9) ** (1 / self.generations)
        self.alpha = (1 - delta) * self.alpha

In [4]:
def Lstm_firefly(firefly):
    print("la fonction objective de l'algorithme de firefly va avoir une luciole en paramère et"
          "retourne le taux de perte de modèle basé sur les paramètre de cette luciole")
    epochs=int(firefly[1])
    filters=int(firefly[0])
    #x_test,y_test=generate_x_test()
    dropout = 0.0
    simple_lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(
        128, input_shape=X_train_w.shape[-2:], dropout=dropout),
    tf.keras.layers.Dense(128),
    tf.keras.layers.Dense(128),
    tf.keras.layers.Dense(1)
    ])
    
    simple_lstm_model.compile(optimizer='rmsprop', loss='mse')



    EVALUATION_INTERVAL = 200
    EPOCHS = epochs

    model_history = simple_lstm_model.fit(train_data, epochs=EPOCHS,
                                          steps_per_epoch=EVALUATION_INTERVAL,
                                          validation_data=val_data, validation_steps=filters)  # ,callbacks=[tensorboard_callback]) #Uncomment this line for tensorboard support


    yhat = simple_lstm_model.predict(X_test_w).reshape(1, -1)[0]
    rmse = sqrt(mean_squared_error(y_test, yhat))

    return rmse


In [ ]:
# save model

In [35]:
print("creation des firefly....;")
population = [Firefly(fireflies[i],allLosses[i],Lstm_firefly)for i in range(len(fireflies))]

creation des firefly....;


In [36]:
print("lancement algorithme de firefly..........")
print("l'algorithme peut prendre un peu de temps merci de patienter")
optimizer=FireflyOptimizer(Lstm_firefly)
brightness,position = optimizer.run_firefly(population)

lancement algorithme de firefly..........
l'algorithme peut prendre un peu de temps merci de patienter
la fonction objective de l'algorithme de firefly va avoir une luciole en paramère etretourne le taux de perte de modèle basé sur les paramètre de cette luciole
Epoch 1/30
200/200 [==============================] - 44s 222ms/step - loss: 433853.1250 - val_loss: 591027.4375
Epoch 2/30
200/200 [==============================] - 42s 210ms/step - loss: 509254.5938 - val_loss: 591018.8750
Epoch 3/30
200/200 [==============================] - 39s 194ms/step - loss: 587535.4375 - val_loss: 597184.8750
Epoch 4/30
200/200 [==============================] - 44s 218ms/step - loss: 450124.6250 - val_loss: 593759.8125
Epoch 5/30
200/200 [==============================] - 44s 218ms/step - loss: 358455.3750 - val_loss: 601606.8125
Epoch 6/30
200/200 [==============================] - 42s 212ms/step - loss: 528525.6250 - val_loss: 620909.2500
Epoch 7/30
200/200 [==============================] - 44s 2

200/200 [==============================] - 41s 206ms/step - loss: 460163.2500 - val_loss: 748950.1875
Epoch 18/29
200/200 [==============================] - 47s 234ms/step - loss: 422771.8438 - val_loss: 705787.0625
Epoch 19/29
200/200 [==============================] - 50s 248ms/step - loss: 433250.0000 - val_loss: 754760.2500
Epoch 20/29
200/200 [==============================] - 38s 191ms/step - loss: 495411.2812 - val_loss: 806803.6250
Epoch 21/29
200/200 [==============================] - 40s 198ms/step - loss: 408570.3750 - val_loss: 735472.6875
Epoch 22/29
200/200 [==============================] - 41s 203ms/step - loss: 457054.1562 - val_loss: 730697.0625
Epoch 23/29
200/200 [==============================] - 39s 197ms/step - loss: 491164.2812 - val_loss: 656473.7500
Epoch 24/29
200/200 [==============================] - 38s 191ms/step - loss: 427784.4688 - val_loss: 677751.8750
Epoch 25/29
200/200 [==============================] - 43s 213ms/step - loss: 529863.7500 - val_loss

Epoch 7/30
200/200 [==============================] - 16s 78ms/step - loss: 484962.4062 - val_loss: 1219148.0000
Epoch 8/30
200/200 [==============================] - 16s 78ms/step - loss: 630273.0000 - val_loss: 1338708.2500
Epoch 9/30
200/200 [==============================] - 16s 79ms/step - loss: 815565.9375 - val_loss: 1305619.3750
Epoch 10/30
200/200 [==============================] - 16s 81ms/step - loss: 764479.9375 - val_loss: 1297882.2500
Epoch 11/30
200/200 [==============================] - 17s 85ms/step - loss: 541802.2500 - val_loss: 1237038.5000
Epoch 12/30
200/200 [==============================] - 16s 79ms/step - loss: 632443.8125 - val_loss: 1274655.6250
Epoch 13/30
200/200 [==============================] - 16s 78ms/step - loss: 529520.9375 - val_loss: 1236529.5000
Epoch 14/30
200/200 [==============================] - 16s 78ms/step - loss: 761013.3750 - val_loss: 1241995.5000
Epoch 15/30
200/200 [==============================] - 16s 79ms/step - loss: 446054.3750 - 

Epoch 27/29
200/200 [==============================] - 18s 92ms/step - loss: 374373.0000 - val_loss: 688979.4375
Epoch 28/29
200/200 [==============================] - 18s 92ms/step - loss: 708285.2500 - val_loss: 759768.6250
Epoch 29/29
200/200 [==============================] - 19s 96ms/step - loss: 406820.6875 - val_loss: 713347.9375


In [41]:
print("l'alghoritme a bien terminé et trouvé la bonne ruesultat")
print("**********************************best parametre :**********************************")
print(brightness,position)

l'alghoritme a bien terminé et trouvé la bonne ruesultat
**********************************best parametre :**********************************
796.4490197497768 [ 6 36]


In [43]:
print("Fiiiiiiiiiiiiiiiiiiiiiiinnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn")
print("lancement d'entrainnement de modèle terminale  en utilisant les meilleur hyperparamètre")
print("..................")
print("on commmence..................")
Lstm_optm(position[1], position[0],True)

Fiiiiiiiiiiiiiiiiiiiiiiinnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn
lancement d'entrainnement de modèle terminale  en utilisant les meilleur hyperparamètre
..................
on commmence..................
Epoch 1/36
200/200 [==============================] - 21s 107ms/step - loss: 427825.0938 - val_loss: 275384.8438
Epoch 2/36
200/200 [==============================] - 24s 122ms/step - loss: 514515.7188 - val_loss: 276025.5000
Epoch 3/36
200/200 [==============================] - 21s 104ms/step - loss: 586954.8750 - val_loss: 278298.7812
Epoch 4/36
200/200 [==============================] - 20s 98ms/step - loss: 451186.1250 - val_loss: 270535.6562
Epoch 5/36
200/200 [==============================] - 18s 92ms/step - loss: 358373.4375 - val_loss: 267063.7812
Epoch 6/36
200/200 [==============================] - 22s 110ms/step - loss: 523298.0312 - val_loss: 330794.1562
Epoch 7/36
200/200 [==============================] - 20s 101ms/step - loss: 484892.5312 - val_loss: 280707.5938
Epoch 8/36
200/

796.1949502615203

In [29]:
import tensorflow as tf 

dropout = 0.0
simple_lstm_model = tf.keras.models.Sequential([
tf.keras.layers.LSTM(
    128, input_shape=X_train_w.shape[-2:], dropout=dropout),
tf.keras.layers.Dense(128),
tf.keras.layers.Dense(128),
tf.keras.layers.Dense(1)
])

simple_lstm_model.compile(optimizer='rmsprop', loss='mse')



EVALUATION_INTERVAL = 200
EPOCHS = 150

model_history = simple_lstm_model.fit(train_data, epochs=EPOCHS,
                                      steps_per_epoch=EVALUATION_INTERVAL,
                                      validation_data=val_data, validation_steps=6)  # ,callbacks=[tensorboard_callback]) #Uncomment this line for tensorboard support


Epoch 1/150
200/200 [==============================] - 35s 176ms/step - loss: 434665.8438 - val_loss: 275927.2812
Epoch 2/150
200/200 [==============================] - 34s 172ms/step - loss: 508440.0938 - val_loss: 274443.4688
Epoch 3/150
200/200 [==============================] - 35s 176ms/step - loss: 585669.7500 - val_loss: 272572.4688
Epoch 4/150
200/200 [==============================] - 33s 163ms/step - loss: 454159.3125 - val_loss: 268529.7500
Epoch 5/150
200/200 [==============================] - 32s 162ms/step - loss: 373458.8438 - val_loss: 274722.3438
Epoch 6/150
200/200 [==============================] - 32s 161ms/step - loss: 513404.1562 - val_loss: 298050.0312
Epoch 7/150
200/200 [==============================] - 33s 165ms/step - loss: 485237.9062 - val_loss: 272424.0312
Epoch 8/150
200/200 [==============================] - 34s 169ms/step - loss: 632445.5000 - val_loss: 358476.4062
Epoch 9/150
200/200 [==============================] - 33s 164ms/step - loss: 819425.812

200/200 [==============================] - 61s 307ms/step - loss: 382833.0000 - val_loss: 270044.4062
Epoch 73/150
200/200 [==============================] - 62s 311ms/step - loss: 451444.9688 - val_loss: 292295.2812
Epoch 74/150
200/200 [==============================] - 62s 309ms/step - loss: 480968.3125 - val_loss: 288799.3438
Epoch 75/150
200/200 [==============================] - 49s 243ms/step - loss: 413426.5938 - val_loss: 292762.1562
Epoch 76/150
200/200 [==============================] - 36s 182ms/step - loss: 514176.1250 - val_loss: 274643.4688
Epoch 77/150
200/200 [==============================] - 22s 108ms/step - loss: 798322.5000 - val_loss: 282151.2188
Epoch 78/150
200/200 [==============================] - 22s 111ms/step - loss: 347619.3125 - val_loss: 271854.2812
Epoch 79/150
200/200 [==============================] - 23s 113ms/step - loss: 689351.5000 - val_loss: 285344.6250
Epoch 80/150
200/200 [==============================] - 22s 109ms/step - loss: 1887663.5000 -

Epoch 143/150
200/200 [==============================] - 22s 110ms/step - loss: 834236.6250 - val_loss: 281065.7188
Epoch 144/150
200/200 [==============================] - 22s 108ms/step - loss: 571699.8125 - val_loss: 281130.1562
Epoch 145/150
200/200 [==============================] - 23s 113ms/step - loss: 1226738.1250 - val_loss: 300595.7500
Epoch 146/150
200/200 [==============================] - 22s 108ms/step - loss: 4237394.5000 - val_loss: 273721.9062
Epoch 147/150
200/200 [==============================] - 22s 110ms/step - loss: 42872816.0000 - val_loss: 280356.6250
Epoch 148/150
200/200 [==============================] - 22s 109ms/step - loss: 3493347.5000 - val_loss: 290365.4688
Epoch 149/150
200/200 [==============================] - 22s 111ms/step - loss: 978682.1875 - val_loss: 268095.2812
Epoch 150/150
200/200 [==============================] - 22s 110ms/step - loss: 677686.2500 - val_loss: 270376.6562


In [32]:
simple_lstm_model.save('C:/Users/lenovo/PycharmProjects/forcoasting/my_model_supliers.h5')